### 환경 설정

In [1]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


.env 로부터 설정 읽어오기

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

openAI API key 가져와서 API 호출

In [2]:
import os
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
  api_key=api_key,
)

### 기본적인 Prompt 구조

Prompt에는 2가지 종류가 있다.
1. 사용자가 LLM한테 실제로 전달하는 Prompt = User Prompt
2. User Prompt 이전에 오는 해당 LLM 모델에 적합한 메타 Prompt = System Prompt

System Prompt
- User Prompt를 전달하기 전에 관련 맥락이나 지침을 설정하는 Prompt
  - 페르소나, 어조 등도 설정 할 수 있음
- System Prompt 예시
  - 출력값 지정 (ex. JSON Formatting)
  - 페르소나 및 어조 설정
  - 외부 정보 주입
  - 모델이 지켜야 할 지침


In [3]:
query = "안녕하세요!"

chat_completion = client.chat.completions.create(
    messages=[
      {
        "role": "system", 
        "content": "You are a helpful assistant."
      },
      {
          "role": "user",
          "content": query,
      }
    ],
    max_tokens=300,
    temperature=0,
    model="gpt-3.5-turbo",
)

In [4]:
chat_completion

ChatCompletion(id='chatcmpl-9a3V7EQboVaU0UYaEv5QGKYEPVITE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요! 무엇을 도와드릴까요?', role='assistant', function_call=None, tool_calls=None))], created=1718380293, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=23, total_tokens=44))

In [5]:
chat_completion.choices[0].message.content

'안녕하세요! 무엇을 도와드릴까요?'

템플릿을 활용해 프롬프트 만들기

In [6]:
from string import Template

prompt_template = Template(f"""
다음 사용자의 요청에 답변해주세요.
사용자: {query}"
답변:
""")

print(prompt_template.substitute(query=query))


다음 사용자의 요청에 답변해주세요.
사용자: 안녕하세요!"
답변:



In [7]:
system_prompt = """
당신은 상담을 해주는 인공지능 에이전트입니다. 친절하고 자세하게 사용자와 대화를 나눠주세요. 필요하면 이모티콘을 사용해도 좋습니다.
"""

### 옵션
- model: GPT 3.5, GPT-4 버전 별로 선택하실 수 있으며, 보통 더 좋은 성능의 모델일 수록 가격도 그만큼 비싼 편
- messages: Prompt, Role, Response/Output
- temperature: 높을 수록 동일한 Prompt에도 매번 다르게 이야기하는 경향이 강함. 0.0으로 셋팅 시 같은 답변으로만 대답
- max_tokens: 
  - 모델마다 입력 및 출력 최대 길이가 다르며, 보통 각 모델 소개 페이지에서 찾을 수 있다.
  - OpenAI API의 경우 입력 최대 길이 확인: https://platform.openai.com/tokenizer

In [8]:
def QnA(query):
  prompt = prompt_template.substitute(query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
            "role": "system",
            "content": system_prompt,
          },
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-3.5-turbo",
      max_tokens=300,
      temperature=0,
  )
  return chat_completion.choices[0].message.content

QnA("안녕하세요! 만나서 반가와요!")

'안녕하세요! 반가워요 😊 어떻게 도와드릴까요?'

In [9]:
QnA("앞으로 잘 부탁드려요!")

'안녕하세요! 반가워요 😊 어떤 일로 저와 대화를 나누고 싶으신가요? 무엇을 도와드릴까요?'

### Stream 살펴보기
- LLM이 문장을 모두 완성하지 않고 각 단어별로 완성되는데로 실시간으로 보여주는 방법

In [4]:
stream = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role': 'user', 'content': '이순신 장군은 어떤 분인가요?'}],
    temperature=0.0,
    stream=True
)

for chunk in stream:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end='')

이순신 장군은 조선시대의 무신이자 미신으로 유명한 역사적 인물로, 16세기 말부터 17세기 초까지 활약한 조선의 장군입니다. 이순신은 일본의 침략을 막기 위해 전쟁을 치른 무공으로서, 해전에서의 뛰어난 전략과 능력으로 유명합니다. 특히, 명량 해전에서 일본의 강력한 함대를 물리치며 역사상 가장 위대한 해전 중 하나로 꼽히고 있습니다. 이순신은 조선의 국보로 인정받는 인물 중 하나로, 조선사의 영웅으로 여겨지고 있습니다.